In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Pull the day care data for Monroe County
df =  pd.read_csv("https://data.ny.gov/api/views/cb42-qumz/rows.csv?accessType=DOWNLOAD&sorting=true")
in_monroe = df.County == "Monroe"
monroe = df[in_monroe]

In [2]:
# Download the shapefile
zipfile = "cb_2018_36_tract_500k.zip"
response = requests.get("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_36_tract_500k.zip")
zfile = open(zipfile, "wb")
zfile.write(response.content)
zfile.close()

# And read it in
tracts = gpd.read_file("zip://" + zipfile)

In [3]:
def in_tract(x, y, tracts):
    point = Point(x, y)
    for index, row in tracts.iterrows():
        if point.within(row["geometry"]):
            return row["AFFGEOID"]

data = list()

for index, row in monroe.iterrows():
    row = row.to_dict()
    row["AFFGEOID"] = in_tract(row["Longitude"], row["Latitude"], tracts)
    data.append(row)

geocoded = pd.DataFrame(data)

geocoded

,Facility ID,Program Type,Region Code,County,Facility Status,Facility Name,Facility Opened Date,License Issue Date,License Expiration Date,Address Omitted,...,Infant Capacity,Toddler Capacity,Preschool Capacity,School Age Capacity,Total Capacity,Program Profile,Latitude,Longitude,Location,AFFGEOID
0,432335,DCC,RRO,Monroe,License,Inspire Learning and Childcare,11/12/2009,11/12/2019,11/11/2023,NaN,...,15,23,53,16,107,https://hs.ocfs.ny.gov/dcfs/Profile/Index/432335,43.233927,-77.926912,"(43.233927, -77.926912)",1400000US36055015200
1,39707,DCC,RRO,Monroe,License,Brockport Child Day Care Center Inc.,09/16/1988,12/01/2019,11/30/2023,NaN,...,16,48,63,55,182,https://hs.ocfs.ny.gov/dcfs/Profile/Index/39707,43.209960,-77.952143,"(43.20996, -77.952143)",1400000US36055015301
2,42575,DCC,RRO,Monroe,License,"The Schoolhouse of Brockport Child Care, Inc.",08/14/1995,08/01/2019,07/31/2023,NaN,...,0,24,47,18,89,https://hs.ocfs.ny.gov/dcfs/Profile/Index/42575,43.195045,-77.933769,"(43.195045, -77.933769)",1400000US36055015400
3,839144,DCC,RRO,Monroe,License,"Brockport Clarkson LC, Inc.",07/11/2019,07/11/2019,07/10/2021,NaN,...,2,11,7,12,32,https://hs.ocfs.ny.gov/dcfs/Profile/Index/839144,43.221061,-77.936801,"(43.221061, -77.936801)",1400000US36055015200
4,745965,DCC,RRO,Monroe,License,"TLFC Child Care and Learning Center, LLC",10/22/2015,10/22/2017,10/21/2021,NaN,...,16,20,22,26,84,https://hs.ocfs.ny.gov/dcfs/Profile/Index/745965,43.220759,-77.942826,"(43.220759, -77.942826)",1400000US36055015200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,669495,FDC,RRO,Monroe,Registration,Franklin Family DayCare,10/31/2013,10/31/2019,10/30/2023,NaN,...,0,0,0,2,8,https://hs.ocfs.ny.gov/dcfs/Profile/Index/669495,43.022596,-77.722165,"(43.02259602, -77.72216479)",1400000US36055013206
716,270970,FDC,RRO,Monroe,Registration,"Thomas, Peggy",10/16/2006,10/16/2018,10/15/2022,NaN,...,0,0,0,2,8,https://hs.ocfs.ny.gov/dcfs/Profile/Index/270970,43.032794,-77.702346,"(43.032794, -77.702346)",1400000US36055013206
717,601510,FDC,RRO,Monroe,Registration,"Wright, Veronica",08/27/2012,08/27/2018,08/26/2022,Y,...,0,0,0,2,8,https://hs.ocfs.ny.gov/dcfs/Profile/Index/601510,NaN,NaN,NaN,None
718,338664,FDC,RRO,Monroe,Registration,"Hart, Carolyn",05/13/2008,05/13/2016,05/12/2020,NaN,...,0,0,0,2,8,https://hs.ocfs.ny.gov/dcfs/Profile/Index/338664,43.031753,-77.668943,"(43.031753, -77.668943)",1400000US36055013206


In [4]:
geocoded.to_excel("Childcare.xlsx", index = False)